In [ ]:
"""
need to create ac_equities_db, after delete this comment
"""
#%matplotlib inline
import zipline
from zipline.api import order_target_percent, symbol, schedule_function, date_rules, time_rules,order, record,set_benchmark
from datetime import datetime
import pytz
from matplotlib import pyplot as plt
import pandas as pd
from collections import OrderedDict

etf = ["SPY","DBC","GLD","IEF","TLT"]
etf_folder_path = "/workspace/beat-the-market-public/test_data/" 

data = OrderedDict()
for sym in etf:
    full_file_path = etf_folder_path + sym + ".csv"
    data[sym] = pd.read_csv(full_file_path, index_col=0, parse_dates=['Date'])
    data[sym] = data[sym][["Open","High","Low","Close","Volume"]]
    #data[sym] = data[sym].resample("1d").mean()
    #data[sym].fillna(method="ffill", inplace=True)
    
    print(data[sym].head())

panel = pd.Panel(data)
panel.minor_axis = ["open","high","low","close","volume"]
panel.major_axis = panel.major_axis.tz_localize(pytz.utc)
print(panel)

def initialize(context):
    context.securities = {
        'SPY': 0.25,
        'TLT': 0.3,
        'IEF': 0.3,
        'GLD': 0.075,
        'DBC': 0.075
    }
    set_benchmark(symbol("SPY"))
    # Schedule rebalance for once a month
    schedule_function(rebalance, date_rules.month_start(), time_rules.market_open())
"""
def handle_data(context, data):
    order(symbol("SPY"), 10)
    record(SPY=data.current(symbol('SPY'), 'price'))
"""
def rebalance(context, data):
    # Loop through the securities
    for sec, weight in context.securities.items():
        sym = symbol(sec)
        # Check if we can trade
        if data.can_trade(sym):
            # Reset the weight
            order_target_percent(sym, weight)

# Set start and end
start = datetime(2007, 1, 1, 8, 15, 12, 0, pytz.UTC)
end = datetime(2018, 12, 31, 8, 15, 12, 0, pytz.UTC)

# Fire off backtest
result = zipline.run_algorithm(
    start=start, # Set start
    end=end, # Set end
    initialize=initialize, # Define startup function
    capital_base=100000, # Set initial capital
    data_frequency = 'daily', # Set data frequency
    data = panel ) # Select bundle

print("Ready to analyze result.")

             Open   High    Low  Close   Volume
Date                                           
1993-01-29  26.32  26.32  26.19  26.30  1003200
1993-02-01  26.32  26.49  26.32  26.49   480500
1993-02-02  26.47  26.56  26.41  26.54   201300
1993-02-03  26.58  26.84  26.56  26.82   529400
1993-02-04  26.92  26.99  26.62  26.94   531500
             Open   High    Low  Close  Volume
Date                                          
2006-02-06  22.37  22.47  21.96  22.05  771500
2006-02-07  21.91  22.05  21.37  21.41  674400
2006-02-08  21.40  21.57  21.26  21.32  335400
2006-02-09  21.47  21.70  21.42  21.52  705900
2006-02-10  21.55  21.58  21.05  21.13  276700
             Open   High    Low  Close    Volume
Date                                            
2004-11-18  44.43  44.49  44.07  44.38   5992000
2004-11-19  44.49  44.92  44.47  44.78  11655300
2004-11-22  44.75  44.97  44.74  44.95  11996000
2004-11-23  44.88  44.92  44.72  44.75   3169200
2004-11-24  44.93  45.05  44.79  45.05   

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.ticker as ticker

# Format for book image
font = {'family' : 'eurostile',
        'weight' : 'normal',
        'size'   : 16}
matplotlib.rc('font', **font)


# Settings
window = 252
year_length = 252

# Copy the columsn we need
df = result.copy().filter(items=['portfolio_value', 'gross_leverage'])

# Function for annualized return
def ann_ret(ts):
    return np.power((ts[-1] / ts[0]), (year_length/len(ts))) -1 

# Function for drawdown
def dd(ts):
    return np.min(ts / np.maximum.accumulate(ts)) - 1

# Get a rolling window
rolling_window = result.portfolio_value.rolling(window)

# Calculate rolling analytics
df['annualized'] = rolling_window.apply(ann_ret)
df['drawdown'] = rolling_window.apply(dd)

# Drop initial n/a values
df.dropna(inplace=True)

# Make a figure
fig = plt.figure(figsize=(12, 12))

# Make the base lower, just to make the graph easier to read
df['portfolio_value'] /= 100

# First chart
ax = fig.add_subplot(411)
ax.set_title('Strategy Results')
ax.plot(df['portfolio_value'],
        linestyle='-',
        color='black',
        label='Equity Curve', linewidth=3.0)

# Set log scale
ax.set_yscale('log')

# Make the axis look nicer
ax.yaxis.set_ticks(np.arange(df['portfolio_value'].min(), df['portfolio_value'].max(), 500 ))
ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%0.0f'))

# Add legend and grid
ax.legend()
ax.grid(False)

# Second chart
ax = fig.add_subplot(412)
ax.plot(df['gross_leverage'],
        label='Strategy exposure'.format(window),
        linestyle='-',
        color='black',
        linewidth=1.0)

# Make the axis look nicer
ax.yaxis.set_ticks(np.arange(df['gross_leverage'].min(), df['gross_leverage'].max(), 0.02 ))
ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%0.2f'))

# Add legend and grid
ax.legend()
ax.grid(True)

# Third chart
ax = fig.add_subplot(413)
ax.plot(df['annualized'],
        label='{} days annualized return'.format(window),
        linestyle='-',
        color='black',
        linewidth=1.0)

# Make the axis look nicer
ax.yaxis.set_ticks(np.arange(df['annualized'].min(), df['annualized'].max(), 0.5 ))
ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%0.1f'))

# Add legend and grid
ax.legend()
ax.grid(True)

# Fourth chart
ax = fig.add_subplot(414)
ax.plot(df['drawdown'],
        label='{} days max drawdown'.format(window),
        linestyle='-',
        color='black',
        linewidth=1.0)

# Make the axis look nicer
ax.yaxis.set_ticks(np.arange(df['drawdown'].min(), df['drawdown'].max(), 0.1 ))
ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%0.1f'))

# Add legend and grid
ax.legend()
ax.grid(True)